In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
%autosave 0

Autosave disabled


# Assignment 7 - Markov Decision Processes

In this exercise, we will implement a simple algorithm to calculate the utility function in a Markov decision process; namely *Value Iteration*. 
We will use the algorithm to solve the simple toy problem from the lecture. We could then transfer it to other domains, for example a problem where Pacman is slightly drunk and cannot rely on his actions being executed properly, so that he has to model his behavior stochastically. However, in the interest of time, this is not part of this assignment.


## Example problem definition

First, we will model the toy problem for the lecture. This is what it looks like, with the reward values for each state filled in:

<img height="200" width="300" src="https://public.lsr.ei.tum.de/team/landsiedel/ai_images/mdp_grid_numpy-0.png"/>

Note that the indexing has changed with respect to the one used in the lecture, to better accomodate the table structures used in this exercise.

All important information about the map will be put in `numpy.arrays` of the size `map_shape`:

In [2]:
map_shape = (3, 4)


The `map_arr` array contains information about which cells are traversible (in the example, all but $(1, 1)$).

In [3]:
map_arr = np.zeros(map_shape, dtype=np.bool)  # initialize an all-zero array
map_arr[:] = True  # This assigns a scalar value to all elements of the array, while keeping its shape.
map_arr[1, 1] = False


The `final_arr` is an array of booleans, which is true for those states which are terminal states (the ones with boxes around their reward values in the image above).

In [4]:
final_arr = np.zeros(map_shape, dtype=np.bool)
final_arr[:] = False
final_arr[0, 3] = True
final_arr[1, 3] = True


Finally, the reward values are set in the `rewards` array.

In [5]:
rewards = np.zeros(map_shape)
rewards[:] = -0.04
rewards[0, 3] = 1
rewards[1, 3] = -1

# print final_arr
# print rewards


The movement model of the agent gives the probability of a state transition, given an action. It is implemented as a dict that maps actions (in this case `u, d, r, l` for up, down, right and left) to a dict of possible moves. The keys of the inner dict are moves, i.e., the difference in the state effected by the transition, and the values are the corresponding probabilities.

In [6]:
from collections import OrderedDict

actions = OrderedDict({'u': {(-1, 0): .8, (0, -1): .1, (0, 1): .1},
                       'l': {(0, -1): .8, (1, 0): .1, (-1, 0): .1},
                       'd': {(1, 0): .8, (0, -1): .1, (0, 1): .1},
                       'r': {(0, 1): .8, (1, 0): .1, (-1, 0): .1}})


If a move would lead to a cell on the playing field that is not traversible (either outside of the map, or a wall), the agent stays at its current position. This utility function is handy to determine if the state resulting from a move is still on the map:

In [7]:
def in_bounds(state, map_shape):
    """utility function to determine if `state` is inside the map."""
    return 0 <= state[0] < map_shape[0] and 0 <= state[1] < map_shape[1]


Furthermore, we want to solve the MDP for an undiscounted cost function, so we set the discount factor $\gamma$ to $1$.

In [8]:
gamma = 1


With this, the complete model of the MDP is known. Now we can do

## Exercise 1: Value Iteration

The value iteration algorithm iteratively updates the utility function $U(s)$ of an MDP model. It is motivated by the Bellman equation:

$$U(s) = R(s) + \gamma \max_{a \in A(s)} \sum_{s^\prime} P(s^\prime \vert s, a) U(s^\prime)$$

Value iteration does not solve this complex, nonlinear system of equations. Instead, for every state $s$, it computes the value of the right-hand side of the Bellman equation, and then assumes this as a new estimate of the true utility function. This process is repeated until the error between old and new estimate is below a threshold.

To start the algorithm, we initialize all utility estimates to zero:

In [9]:
# initialize utilities
def init_utils(map_shape, rewards):
    """initialize all utilities to zero, or to the rewards for final states"""
    utilities = np.zeros(map_shape)
    utilities[final_arr] = rewards[final_arr]

    return utilities


Now, your task is to implement the utility update step by computing the right-hand side of the Bellman equation above once for every state:

*(Note: This is the most demanding exercise of this assignment, the rest will be mostly evaluation and reuse of code from this exercise)*

*Hint:* Note that we do not yet use the just calculated new utility values of states when we compute the values of neighbouring states, but update all the values only once after having calculated the new value for all states. 

In [10]:
def update_utils(utilities, map_shape, map_arr, rewards, final_arr, actions, gamma):
    """run one single step of value iteration"""
    new_utilities = np.zeros(map_shape)
    new_utilities[final_arr] = rewards[final_arr]

    """***Your Code here ***"""
    for i in xrange(map_shape[0]):
        for j in xrange(map_shape[1]):
            if final_arr[i, j] or not map_arr[i, j]:
                continue

            templeList = []
            for a in actions.keys():
                temp = 0
                for b in actions[a].keys():
                    mm, nn = b
                    if in_bounds((mm + i, nn + j), map_shape) and map_arr[(mm + i, nn + j)]:
                        temp += actions[a][b] * utilities[mm + i, nn + j]
                    else:
                        temp += actions[a][b] * utilities[i, j]
                templeList.append(temp)

            new_utilities[i, j]  = rewards[i, j] + gamma * max(templeList)
    # This copies the values from new_utilities to utilities
    utilities[:] = new_utilities[:]
    # strictly, there is no return value needed, since `utilities` is
    # also updated in the calling function, but leave this in for
    # the grading to work!
    return utilities

from copy import deepcopy
from numpy.linalg import norm
utilities = init_utils(map_shape, rewards)
pre = deepcopy(utilities)
error = 10
while error > 10e-4:
    utilities = update_utils(utilities, map_shape, map_arr, rewards, final_arr, actions, gamma)
    error = norm(utilities - pre)
    pre = deepcopy(utilities)

You can visualize the utilities after each step as a color image with this command:

```
plt.imshow(utilities, interpolation='nearest')
```


## Exercise 2: Evaluating the error

Now run a few steps of value iteration on the problem given above. The values should quickly converge to those given in the lecture. Compute the Euclidean distance between old and new utility estimates for each step, and visualize them. How many iterations of the algorithms have to be run so that the distance between estimates goes below $10^{-4}$? Determine this number of necessary iterations for the problem given in this notebook; your `min_num_iterations` function should then return this very number.

**Note:**
To calculate the error, you need to keep track of the utilities before updating them in each step. Since the implementation of the `update_utils` function as given above changes the value of the `utilities` variable also in the calling function (like a C++ *call by reference*), a copy of the old values can be made like so:

```
bkp_utilities = utilities.copy()
update_utils(utilities, map_shape, map_arr, rewards, final_arr, actions, gamma)
# compute distance between `utilities` and `bkp_utilities`, etc.

# check convergence using this command to plot the errors
plt.plot(distances)
```

In [11]:
def min_num_iterations():
    """
    return the minimum number of iterations of value iteration
    needed on the problem given above so that the Euclidean
    distance between utility estimates is smaller than 10**-4
    """
    return 21  # just change this


## Exercise 3: Get strategy from utilities

Now that we have a good estimate of the utility function, we want to use it to decide on actions. Implement the following function, which extracts the optimal policy for the utility function given as an argument. You should be able to reuse most of your value iteration code for this exercise.

The return value of the function should be a numpy array of single characters, where the entry for traversible and nonterminal states should be the key of the optimal action in the movement model (see above), and `'x'` for all other states.

In [12]:
def get_strategy(utilities, map_shape, map_arr, final_arr, actions):
    strategy = np.zeros(map_shape, dtype=np.character)

    """*** Your code here ***"""
    import operator
    for i in xrange(map_shape[0]):
        for j in xrange(map_shape[1]):
            if final_arr[i, j] or not map_arr[i, j]:
                strategy[i, j] = 'x'
                continue

            valueList = {}
            for a in actions.keys():
                temp = 0
                for b in actions[a]:
                    mm, nn = b
                    if in_bounds((mm + i, nn + j), map_shape) and map_arr[(mm + i, nn + j)]:
                        temp += actions[a][b] * utilities[mm + i, nn + j]
                    else:
                        temp += actions[a][b] * utilities[i, j]
                valueList[a] = temp
            
            action = max(valueList.iteritems(), key=operator.itemgetter(1))[0]
            strategy[i, j] = action

    return strategy


In [13]:
strategy = get_strategy(utilities, map_shape, map_arr, final_arr, actions)
print strategy

[['r' 'r' 'r' 'x']
 ['u' 'x' 'u' 'x']
 ['u' 'l' 'l' 'l']]
